In [27]:
import pandas as pd

path = '../C14NL-PoS-annotated/TEST.tsv'
TEST_DF = pd.read_csv(path, sep="\t", encoding = "UTF-8", header=None)


from string import punctuation

def clean_words(word):
  word = word.casefold()
  word = word.translate(str.maketrans('', '', punctuation))
  return word

# print(TRAIN[0])
TEST_DF[0] = TEST_DF[0].apply(clean_words)

TEST_DF.head()

,0,1,2,3,4
0,wie,O,688,H072p40001,140
1,suster,O,688,H072p40001,140
2,lisebette,B-PERS,688,H072p40001,140
3,bi,O,688,H072p40001,140
4,den,O,688,H072p40001,140


In [37]:
X = []

grouped = TEST_DF.groupby(3)

for group in grouped.groups:
    df = grouped.get_group(group)
    X.append(' '.join(list(df[0])))


grouped = TEST_DF.groupby(3)

y = []

for group in grouped.groups:
    df = grouped.get_group(group)
    df[1] = ['ENT' if x != 'O' or x == 'B-MONEY' or x == 'I-MONEY' else 'O' for x in df[1] ]
    y.append(list(df[1]))

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_21908\514411928.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1] = ['ENT' if x != 'O' or x == 'B-MONEY' or x == 'I-MONEY' else 'O' for x in df[1] ]


In [38]:
from typing import List
from pie_extended.cli.utils import get_tagger

model_name = "dum"
tagger = get_tagger(model_name, batch_size=256, device="cpu")

sentences: List[str] = [body[1]]
# Get the main object from the model (: data iterator + postprocesor
from pie_extended.models.dum.imports import get_iterator_and_processor

def get_ents_from_pie(text):
    sentences: List[str] = [text]
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        thing = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)

    lst = []    
    for dct in thing:
        if dct['pos'] == 'n(prop)':
            lst.append('ENT')
        else:
            lst.append('O')
    
    return lst

get_ents_from_pie(X[0])

print(X[0])

2022-04-27 16:07:18,080 : Model C:\Users\Gebruiker\anaconda3\lib\site-packages\pie_extended\downloads\dum\lemma.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.
2022-04-27 16:07:19,296 : Model C:\Users\Gebruiker\anaconda3\lib\site-packages\pie_extended\downloads\dum\pos.tar was serialized with a previous version of `pie`. This might result in issues. Model commit is 050815e, whereas current `pie` commit is 9ce3e8f.


wie suster lisebette bi den ghedoghe van gode abdesse va sinte clare typre kenne dat jc hebbe ontfanghen van der steide va ypre in de name en ouer suster kateline smisen nonne in onsen clooster xxiiij lb par als va de paiemente dat haer viel de xvsten dach va lau maent va de welke paimente voors en van alle verleidene paiemente en al dat atouchieren mach omme cause van der vors lijf rente houdic mi wel vernouch in de name en houer suster kateline voors in kennesse der wareide so hebbe jc abdesse voors deise lre gheseighelt met minen zeighele hute hanghende int jaer nos heren als men screef mccc iiijxx en xix den xvsten dach van lau maent


In [39]:
y_true = []

for doc in y:
    y_true.extend(doc)

y_pred = []

for doc in X:
    ents = get_ents_from_pie(doc)
    # print(ents)
    y_pred.extend(ents)


from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         ENT       0.84      0.42      0.56       941
           O       0.89      0.98      0.93      4309

    accuracy                           0.88      5250
   macro avg       0.87      0.70      0.74      5250
weighted avg       0.88      0.88      0.86      5250

